Long Conditions
*   Close Price > EMA 21 and
*   Williams%R cross abovce -80

Long Exit
*   ADX >=25 and Close Price<EMA21 or
*   ADX <25 and Williams %R cross below-20

Short Condition
*   Close Price < EMA 21 and
*   Willaim %R cross below -20

Short Exit
*   ADX>=25 and Close Price> EMA21 or
*   ADX <25 and William %R cross above-80

Control maximum lot at 40
Stop loss at 20%
ignore opposite signals until current position is close


In [1]:
!pip install vectorbt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.6 MB/s eta 0:00:00


In [2]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=9811e299a01cdc037e49ff499ede7a4787a0abf7ff520922f8fbab869ed6f0ee
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import pandas_ta as ta
import warnings
from datetime import datetime
import time
import plotly.figure_factory as ff
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

In [4]:
symbols = ['USDJPY=X']
start_date = '1996-10-30'
end_date = '2001-10-30'
price = vbt.YFData.download(symbols,start=start_date,end=end_date,interval='1D',missing_index='drop')
price = price.loc[(price.wrapper.index >= start_date) & (price.wrapper.index < end_date)]

In [5]:
df = pd.DataFrame()
df = price.data[symbols[0]]
df.drop(['Volume','Dividends','Stock Splits'],axis=1,inplace=True)

In [6]:
df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')

Video Minute 1.59

In [7]:
EMA = ta.sma(df.Close,length=21)
William = ta.willr(high=df.High,low=df.Low,close=df.Close,length=14)
ADX = ta.adx(high=df.High,low=df.Low,close=df.Close,length=21)

In [24]:
Williams = pd.DataFrame(index=df.index)
Williams['Cross_80_Down'] = William < -80
Williams['Signal'] = Williams['Cross_80_Down']*1
Williams['Signal_1'] = Williams['Signal'].shift(1)
W_Signal = []

for x in np.arange(0,len(Williams)):
  if (Williams.Signal_1[x] == 1) and (Williams.Signal[x] ==0):
      W_Signal.append(True)
  else:
      W_Signal.append(False)

Williams['W_Signal'] = W_Signal # Cross above -80
Williams.loc[Williams.W_Signal == True, 'Marker_Position'] = df.High*1.02


In [25]:
Williams[Williams['W_Signal']==True]

,Cross_80_Down,Signal,Signal_1,W_Signal,Marker_Position
Date,,,,,
1996-11-22,False,0,1.0,True,113.933997
1997-02-26,False,0,1.0,True,125.072403
1997-03-03,False,0,1.0,True,123.756602
1997-05-06,False,0,1.0,True,128.265000
1997-05-21,False,0,1.0,True,118.656602
...,...,...,...,...,...
2001-08-21,False,0,1.0,True,122.971198
2001-08-28,False,0,1.0,True,122.695801
2001-09-03,False,0,1.0,True,122.032799


In [40]:
#Setting Layout
layout = go.Layout(
    title="Testing William R",
    plot_bgcolor="white", #Sets background color to white
    legend=dict(
        #Adjust click behavior
        itemclick="toggleothers",
        itemdoubleclick="toggle"),

    hovermode ='x',
    hoverdistance = 100, #Distance to show hover label of data point
    spikedistance = 1000, #Distance to show spike

    xaxis=dict(
        title="time",
        linecolor="black", #Sets color of X-axis line
        showspikes = True, # Show spike line for x-axis
        spikethickness = 2,
        spikedash = "dot",
        spikecolor = "#999999",
        spikemode = "across",
        showgrid=False # Removes X-axis grid lines
    ),
    yaxis=dict(
        domain=[0.4,1],
        title="Price",
        linecolor="black", #Sets color of Y-axis line
        showgrid=False, # Removes Y-axis grid lines
    ),
    yaxis2=dict(
        domain=[0,0.3],
        anchor="x2",
        title="William",
        linecolor="black", #Sets color of Y-axis line
        showgrid=False, #Removes Y-axis grid lines
    ))

data = []
#Candlestickdata
TS = go.Candlestick(x=df.index,
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close'],
                    increasing_line_color = 'Green',decreasing_line_color = 'Red')
data.append(TS)

#Indicator Part Lower Part
#RSI
date_generated = pd.date_range(df.index[0], periods=len(df))
W = go.Scatter(x=df.Open.index,
               y=William,
               mode='lines', name='wWilliam',marker=dict(color = 'Red'),opacity=0.5,
               yaxis="y2",xaxis='x1')

W_l_line = go.Scatter(x=df.index,
                      y=pd.Series(-80, index =date_generated),
                      mode='lines', name= '-80 William line',marker=dict(color = 'Black'),opacity=0.5,
                      yaxis="y2",xaxis='x1')

W_u_line = go.Scatter(x=df.index,
                      y=pd.Series(-20, index = date_generated),
                      mode='lines', name='-20 William line',marker=dict(color = 'Black'),opacity=0.5,
                      yaxis="y2",xaxis='x1')

data.append(W)
data.append(W_l_line)
data.append(W_u_line)

Short_Signal = go.Scatter(
    x=df.index,
    y=Williams.Marker_Position ,
    mode='markers', marker_symbol = "triangle-down", marker_size=10,
    name='short_signal',marker=dict(color='red'),opacity=0.5)

data.append(Short_Signal)

fig = go.Figure(data=data, layout=layout)

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show(config={"displayModeBar": True, "showTips": False})

In [53]:
Williams = pd.DataFrame(index=df.index)
Williams['Cross_80_Down'] = William < -80
Williams['Cross_20_Above'] = William > -20
Williams['Signal_80'] = Williams['Cross_80_Down']*1
Williams['Signal_20'] = Williams['Cross_20_Above']*1
Williams['Signal_1'] = Williams['Signal_80'].shift(1)
Williams['Signal_2'] = Williams['Signal_20'].shift(1)
W_Signal_80 = []
W_Signal_20 = []

for x in np.arange(0,len(Williams)):
  if (Williams.Signal_1[x] == 1) and (Williams.Signal_80[x]==0):
      W_Signal_80.append(True)
  else:
      W_Signal_80.append(False)

for x in np.arange(0,len(Williams)):
  if (Williams.Signal_2[x] == 1) and (Williams.Signal_20[x]==0):
      W_Signal_20.append(True)
  else:
      W_Signal_20.append(False)

Williams['W_Signal_80'] = W_Signal_80 #Cross -80 Above
Williams['W_Signal_20'] = W_Signal_20 #Cross -20 Down

In [54]:
Williams[Williams.W_Signal_20==True]

,Cross_80_Down,Cross_20_Above,Signal_80,Signal_20,Signal_1,Signal_2,W_Signal_80,W_Signal_20
Date,,,,,,,,
1996-12-03,False,False,0,0,0.0,1.0,False,True
1996-12-27,False,False,0,0,0.0,1.0,False,True
1997-01-02,False,False,0,0,0.0,1.0,False,True
1997-01-06,False,False,0,0,0.0,1.0,False,True
1997-01-15,False,False,0,0,0.0,1.0,False,True
...,...,...,...,...,...,...,...,...
2001-09-10,False,False,0,0,0.0,1.0,False,True
2001-10-07,False,False,0,0,0.0,1.0,False,True
2001-10-14,False,False,0,0,0.0,1.0,False,True


In [64]:
EMA_Cross_above = df.Close>EMA
EMA_Cross_below = df.Close<EMA

EMA_Cross_above_1 = ta.tsignals(EMA_Cross_above,asbool=True,append=True)
EMA_Cross_below_1 = ta.tsignals(EMA_Cross_below,asbool=True,append=True)

Signal_EMA_Cross_above = EMA_Cross_above_1.TS_Entries
Signal_EMA_Cross_below_1 = EMA_Cross_below_1.TS_Entries

In [65]:
Signal = pd.DataFrame(index=df.index)
Long_Entry = []

for x in np.arange(0,len(Signal_EMA_Cross_above)):
    if (Signal_EMA_Cross_above[x] == True) and (Williams.W_Signal_80[x]==True):
        Long_Entry.append(True)
    else:
        Long_Entry.append(False)

Signal["Long_Entry"] = Long_Entry

In [66]:
Signal[Signal.Long_Entry==True]

,Long_Entry
Date,
1999-12-24,True
2000-01-04,True
2000-10-18,True


In [68]:
Short_Entry = []
Long_Exit = []
Short_Exit = []

for x in np.arange(0,len(Signal_EMA_Cross_below_1)):
    if (Signal_EMA_Cross_below_1[x] == True) and  (Williams.W_Signal_20[x]==True):
        Short_Entry.append(True)
    else:
        Short_Entry.append(False)

for x in np.arange(0,len(Signal_EMA_Cross_below_1)):
    if ((ADX.ADX_21[x]>=25) and (Signal_EMA_Cross_below_1[x] == True)) or ((ADX.ADX_21[x]<25) and (Williams.W_Signal_20[x]==True)):
      Long_Exit.append(True)
    else:
      Long_Exit.append(False)

for x in np.arange(0,len(Signal_EMA_Cross_above)):
    if ((ADX.ADX_21[x]>25) and (Signal_EMA_Cross_above[x] == True)) or ((ADX.ADX_21[x]<25) and(Williams.W_Signal_80[x]==True)):
        Short_Exit.append(True)
    else:
        Short_Exit.append(False)

Signal["Short_Entry"] = Short_Entry
Signal["Long_Exit"] = Long_Exit
Signal["Short_Exit"] = Short_Exit

In [69]:
port = vbt.Portfolio.from_signals(df.Close,
                                  entries = Signal.Long_Entry, # Long Entry
                                  exits = Signal.Long_Exit, # Long Exit
                                  short_entries = Signal.Short_Entry, #Short Entry
                                  short_exits = Signal.Short_Exit, #Short Exit
                                  max_size = 40, #Define max size
                                  freq = 'D',
                                  sl_stop = 0.2, #20% of purchasing price
                                  upon_opposite_entry='ignore', #ignore opposite signals until current position is close
                                  init_cash = 10000,
                                  fees = 0.0025,
                                  slippage = 0.0025)

port.plot().show()

In [70]:
port.stats()

Start                                 1996-10-30
End                                   2001-10-29
Period                        1290 days 00:00:00
Start Value                              10000.0
End Value                            9384.765087
Total Return [%]                       -6.152349
Benchmark Return [%]                    6.752495
Max Gross Exposure [%]              59251.580465
Total Fees Paid                       227.817998
Max Drawdown [%]                        7.212334
Max Drawdown Duration         1193 days 00:00:00
Total Trades                                  10
Total Closed Trades                           10
Total Open Trades                              0
Open Trade PnL                               0.0
Win Rate [%]                                20.0
Best Trade [%]                           1.79222
Worst Trade [%]                        -3.672155
Avg Winning Trade [%]                   1.293233
Avg Losing Trade [%]                   -1.958433
Avg Winning Trade Du

In [71]:
port.order_records

array([( 0, 0,   97, 40., 121.59525152, 12.15952515, 1),
       ( 1, 0,  123, 40., 125.44282225, 12.54428222, 0),
       ( 2, 0,  210, 40., 118.43317835, 11.84331783, 1),
       ( 3, 0,  238, 40., 121.01177408, 12.10117741, 0),
       ( 4, 0,  281, 40., 126.78224848, 12.67822485, 1),
       ( 5, 0,  283, 40., 129.53303173, 12.95330317, 0),
       ( 6, 0,  616, 40., 118.47307165, 11.84730717, 1),
       ( 7, 0,  630, 40., 119.13709633, 11.91370963, 0),
       ( 8, 0,  758, 40., 105.39585365, 10.53958537, 1),
       ( 9, 0,  761, 40., 106.55572592, 10.65557259, 0),
       (10, 0,  784, 40., 104.4183003 , 10.44183003, 1),
       (11, 0,  786, 40., 104.97177408, 10.49717741, 0),
       (12, 0,  808, 40., 103.07704969, 10.30770497, 0),
       (13, 0,  821, 40., 105.44572409, 10.54457241, 1),
       (14, 0,  886, 40., 105.485625  , 10.5485625 , 1),
       (15, 0,  914, 40., 107.44795031, 10.74479503, 0),
       (16, 0, 1022, 40., 108.60082684, 10.86008268, 0),
       (17, 0, 1030, 40., 108.5

In [72]:
port.drawdowns.plot().show()

In [73]:
port.exit_trades.plot().show()